In [2]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.optimize as opt

In [3]:
test_matrix = np.array([[1, 2], [3, 4]])
big_test_matrix = la.block_diag(test_matrix, test_matrix)
readable_big_test_matrix = pd.DataFrame(big_test_matrix)
print(type(big_test_matrix))

<class 'numpy.ndarray'>


In [4]:
def kron(A, B):
    return la.kron(A, B)

In [5]:
def kronsum(A, B):
    if A.shape[0] != A.shape[1]:
        raise ValueError('A is not square')

    if B.shape[0] != B.shape[1]:
        raise ValueError('B is not square')
    
    L = kron(A, np.eye(B.shape[0]))
    R = kron(B, np.eye(A.shape[0]))
    
    return L+R

In [6]:
print(kron(big_test_matrix[:2, :2], big_test_matrix[2:4, 2:4]))

[[ 1  2  2  4]
 [ 3  4  6  8]
 [ 3  6  4  8]
 [ 9 12 12 16]]


In [7]:
n = 3

In [8]:
# Входной BMAP
matrD_0 = np.matrix([[-0.2, 0.6], [0.1, -0.4]])
matrD = np.matrix([[85, 0.99], [0.2, 2.54]])
matrD_k = [matrD_0]

W_ = matrD_0.shape[0]
W = W_ - 1

q = 0.8
for k in range(1, n+1):
    matrD_k.append(matrD * (q ** (k-1)) * (1 - q) / (1 - q ** 3))

print(matrD_0)

[[-0.2  0.6]
 [ 0.1 -0.4]]


In [9]:
# Поток поломок MMAP
matrH0 = np.matrix([[-8.110725, 0], [0, -0.26325]])
matrH1 = np.matrix([[8.0568, 0.053925], [0.146625, 0.116625]])
V_ = matrH1.shape[0]
V = V_ - 1
matrH = matrH0 + matrH1

In [10]:
# Поток обслуживания PH1
beta1 = np.matrix([1, 0])
matrS1 = np.matrix([[-20, 20], [0, -20]])
M1 = matrS1.shape[0]
M1_ = M1 + 1
M1_e = np.matrix([[1], [1]])
matrS1_0 = - matrS1 * M1_e

In [11]:
# Поток обслуживания PH2
beta2 = np.matrix([1, 0])
matrS2 = np.matrix([[-15, 15], [0, -15]])
M2 = matrS2.shape[0]
M2_ = M2 + 1
M2_e = np.matrix([[1], [1]])
matrS2_0 = - matrS2 * M2_e

In [12]:
matrS_w = kron(matrS1_0 * beta1, M2_e * beta2) + kron(M1_e * beta1, matrS2_0 * beta2)

In [13]:
# Поток переключения с прибора-1 на прибор-2
alpha1 = np.matrix([1, 0])
matrA1 = np.matrix([[-20, 20], [0, -20]])
L1 = matrA1.shape[0]
L1_ = L1 + 1
L1_e = np.matrix([[1], [1]])
matrA1_0 = - matrA1.dot(L1_e)

In [14]:
# Поток переключения с прибора-2 на прибор-1
alpha2 = np.matrix([1, 0])
matrA2 = np.matrix([[-15, 15], [0, -15]])
L2 = matrA2.shape[0]
L2_ = L2 + 1
L2_e = np.matrix([[1], [1]])
matrA2_0 = - matrA2.dot(L2_e)

In [15]:
# Поток ремонта PH
tau = np.matrix([0.05, 0.95])
matrT = np.matrix([[-1.86075, 0], [0, -146.9994]])
T_e = np.matrix([[1],[1]])
matrT0 = - matrT.dot(T_e)

R = matrT.shape[0]
R_ = R + 1

In [16]:
a = W_ * V_

In [104]:
# Q~0
block00 = kronsum(matrD_k[0], matrH0)
block03 = kron(kron(kron(np.eye(W_), matrH1), tau), alpha1)
block10 = kron(np.eye(a), matrA2_0)
block11 = kronsum(kronsum(matrD_k[0], matrH0), matrA2)
block12 = kron(kron(kron(np.eye(W_), matrH1), tau), L2_e)
block21 = kron(kron(np.eye(a), matrT0), alpha2)
block22 = kronsum(kronsum(matrD_k[0], matrH), matrT)
block30 = kron(kron(np.eye(a), matrT0), L1_e)
block32 = kron(kron(np.eye(a), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

print(block00.shape[1] + block11.shape[1] + block22.shape[1] + block33.shape[1])

matrQw_0 = np.bmat([[block00, block01, block02, block03],
                    [block10, block11, block12, block13], 
                    [block20, block21, block22, block23],
                    [block30, block31, block32, block33]])
pd.DataFrame(matrQw_0)

36


,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-8.310725,-0.000000,0.60000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.051229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.000000,-8.310725,0.00000,0.60000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.110794,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.100000,0.000000,-0.66325,-0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.402840,0.000000,7.653960,0.000000,0.002696,0.000000,0.051229,0.000000
3,0.000000,0.100000,-0.00000,-0.66325,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007331,0.000000,0.139294,0.000000,0.005831,0.000000,0.110794,0.000000
4,0.000000,0.000000,0.00000,0.00000,-23.310725,-0.000000,-0.000000,-0.000000,15.60000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,15.000000,0.000000,0.00000,0.00000,-0.000000,-23.310725,-0.000000,-0.000000,0.00000,15.60000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.00000,0.00000,-0.000000,-0.000000,-23.310725,-0.000000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,15.000000,0.00000,0.00000,-0.000000,-0.000000,-0.000000,-23.310725,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.00000,0.00000,0.100000,0.000000,0.000000,0.000000,-15.66325,-0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,15.00000,0.00000,0.000000,0.100000,0.000000,0.000000,-0.00000,-15.66325,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [105]:
# Q~k
matrQw_k = [matrQw_0]
for i in range(1, n+1):
    block0 = kron(kron(matrD_k[i], np.eye(V_)), beta1)
    block1 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(L2))
    block2 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(R))
    block3 = kron(kron(matrD_k[i], np.eye(V_)), np.eye(R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQw_k.append(matr_temp)
pd.DataFrame(matrQw_k[1])

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,34.836066,0.0,0.000000,0.0,0.405738,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.0,34.836066,0.0,0.000000,0.0,0.405738,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.081967,0.0,0.000000,0.0,1.040984,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.0,0.081967,0.0,0.000000,0.0,1.040984,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,34.836066,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,34.836066,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.081967,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.081967,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
# Q^0
block0 = kron(np.eye(a), matrS1_0)
block1 = kron(kron(np.eye(a), matrS2_0), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQv_0 = la.block_diag(block0, block1, block2, block3)
pd.DataFrame(matrQv_0)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Q_0
block0 = kron(np.eye(a), matrS1_0 * beta1)
block1 = kron(kron(np.eye(a), matrS2_0 * beta2), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0 * beta2), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQ_0 = la.block_diag(block0, block1, block2, block3)
pd.DataFrame(matrQ_0)
print(kronsum(kronsum(kronsum(matrS1, matrS2), matrH0), matrD_0).shape)

(16, 16)


In [21]:
# Q_1
block00 = kronsum(kronsum(matrD_k[0], matrH0), matrS1)
block03 = kron(kron(kron(kron(np.eye(W_), matrH1), M1_e), tau), alpha1)
block10 = kron(kron(kron(np.eye(a), M2_e), beta1), matrA2_0)
block11 = kronsum(kronsum(kronsum(matrD_k[0], matrH0), matrS2), matrA2)
block12 = kron(kron(kron(kron(np.eye(W_), matrH1), np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(a), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrS2), matrT)
block30 = kron(kron(kron(np.eye(a), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(a), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

matrQ_1 = np.bmat([[block00, block01, block02, block03], 
                   [block10, block11, block12, block13], 
                   [block20, block21, block22, block23], 
                   [block30, block31, block32, block33]])
pd.DataFrame(matrQ_1)

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,-28.310725,-0.000000,-0.000000,-0.000000,20.60000,0.00000,0.00000,0.00000,0.000000,0.000000,...,0.051229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.000000,-28.310725,-0.000000,-0.000000,0.00000,20.60000,0.00000,0.00000,0.000000,0.000000,...,0.051229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.000000,-0.000000,-28.310725,-0.000000,0.00000,0.00000,20.60000,0.00000,0.000000,0.000000,...,0.110794,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-0.000000,-0.000000,-0.000000,-28.310725,0.00000,0.00000,0.00000,20.60000,0.000000,0.000000,...,0.110794,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.100000,0.000000,0.000000,0.000000,-20.66325,-0.00000,-0.00000,-0.00000,0.000000,0.000000,...,0.000000,0.000000,0.402840,0.000000,7.653960,0.000000,0.002696,0.000000,0.051229,0.000000
5,0.000000,0.100000,0.000000,0.000000,-0.00000,-20.66325,-0.00000,-0.00000,0.000000,0.000000,...,0.000000,0.000000,0.402840,0.000000,7.653960,0.000000,0.002696,0.000000,0.051229,0.000000
6,0.000000,0.000000,0.100000,0.000000,-0.00000,-0.00000,-20.66325,-0.00000,0.000000,0.000000,...,0.000000,0.000000,0.007331,0.000000,0.139294,0.000000,0.005831,0.000000,0.110794,0.000000
7,0.000000,0.000000,0.000000,0.100000,-0.00000,-0.00000,-0.00000,-20.66325,0.000000,0.000000,...,0.000000,0.000000,0.007331,0.000000,0.139294,0.000000,0.005831,0.000000,0.110794,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,-38.310725,-0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,15.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,-0.000000,-38.310725,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
matrQ_k = [matrQ_0, matrQ_1]
for k in range(2, n+1):
    block0 = kron(matrD_k[k-1], np.eye(V_ * M1))
    block1 = kron(matrD_k[k-1], np.eye(V_*M2*L2))
    block2 = kron(matrD_k[k-1], np.eye(V_*M2*R))
    block3 = kron(matrD_k[k-1], np.eye(V_*R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQ_k.append(matr_temp)

In [ ]:
# Поиск матрицы G
def algo1(matrG):
    res = matrQ_k[0]
    for k in range(1, n+1):
        res += matrQ_k[k].dot(np.linalg.matrix_power(matrG, k))
    return res

initial_matrG = np.eye(matrQ_k[1].shape[0])
matrG = opt.newton_krylov(algo1, initial_matrG, iter=1000000, verbose=False)
pd.DataFrame(matrG)

In [34]:
temp_sum = matrQ_k[1]
for k in range(2, n+1):
    temp_sum += matrQ_k[k].dot(np.linalg.matrix_power(matrG, k-1))
matrG_0 = la.inv(temp_sum)
matrG_0 = matrG_0.dot(matrQv_0)
pd.DataFrame(matrG_0)

(56, 56)


,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.023837,0.000000,-0.003037,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.023837,0.000000,-0.003037,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000320,0.000000,-0.909634,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000320,0.000000,-0.909634,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
matrQ_il = []
for i in range(0, n+1):
    matrQ_il.append([])
    if i == 0:
        for l in range(0, n+1):
            temp_matr = matrQw_k[l]
            for k in range(l+1, n+1):
                mult_matr = matrQw_k[k]
                for kk in range(k-1, l-1, -1):
                    if kk == 0:
                        mult_matr = mult_matr.dot(matrG_0)
                    else:
                        mult_matr = mult_matr.dot(matrG)
                temp_matr += mult_matr
            matrQ_il[i].append(temp_matr)
    else:
        for l in range(i, n+1):
            temp_matr = matrQ_k[l-i]
            for k in range(l+1, n+1):
                mult_matr = matrQ_k[k-i]
                for kk in range(k-1, l-1, -1):
                    if kk == 0:
                        mult_matr = mult_matr.dot(matrG_0)
                    else:
                        mult_matr = mult_matr.dot(matrG)
                temp_matr += mult_matr
            matrQ_il[i].append(temp_matr)
    
pd.DataFrame(matrQ_il[0][0])

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-8.310725,0.000000,0.60000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.051229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,-8.310725,0.00000,0.60000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.110794,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.100000,0.000000,-0.66325,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.402840,0.000000,7.653960,0.000000,0.002696,0.000000,0.051229,0.000000
3,0.000000,0.100000,0.00000,-0.66325,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007331,0.000000,0.139294,0.000000,0.005831,0.000000,0.110794,0.000000
4,0.000000,0.000000,0.00000,0.00000,-23.310725,0.000000,0.000000,0.000000,15.60000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,15.000000,0.000000,0.00000,0.00000,0.000000,-23.310725,0.000000,0.000000,0.00000,15.60000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,-23.310725,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,15.000000,0.00000,0.00000,0.000000,0.000000,0.000000,-23.310725,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.00000,0.00000,0.100000,0.000000,0.000000,0.000000,-15.66325,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,15.00000,0.00000,0.000000,0.100000,0.000000,0.000000,0.00000,-15.66325,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [126]:
matrPhi_0 = np.eye(matrQ_il[0][0].shape[0])
matrPhi_l = [matrPhi_0]
for l in range(1, n+1):
    temp_matr = np.zeros(matrQ_il[0][l].shape)
    print(temp_matr.shape)
    for i in range(0, l):
        # print(matrPhi_l[i].dot(matrQ_il[i][l]).dot(la.inv(-matrQ_il[l][l])).shape)
        temp_matr += (matrPhi_l[i].dot(matrQ_il[i][l])).dot(la.inv(-matrQ_il[l][l]))
    matrPhi_l.append(temp_matr)
pd.DataFrame(matrPhi_l[1])

(36, 56)
(36, 56)


IndexError: list index out of range